In [1]:
import sklearn
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from joblib import dump, load
from sklearn.decomposition import PCA
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.constraints import max_norm

from keras import backend as K
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cmocean
import colorcet as cc

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
cartopy.config["data_dir"] = '../cartopy_shapefiles'
from pathlib import Path
import gsw
import time

pathfig = '/home1/datawork/epauthen/Fig_Product/All_depth/'
path_model = "/home/datawork-lops-bluecloud/osnet/models/all_depth500"

In [2]:
%%time
#Load dataset and fill NaN with O
x = xr.open_dataset("/home/datawork-lops-bluecloud/osnet/data_cora_raw/CORA_alldepth_bathy500_1993-2019.nc")
x = x.fillna(0)
x

CPU times: user 273 ms, sys: 371 ms, total: 644 ms
Wall time: 876 ms


<xarray.Dataset>
Dimensions:          (N_PROF: 87700, depth: 114)
Coordinates:
    longitude        (N_PROF) float64 ...
    latitude         (N_PROF) float64 ...
  * depth            (depth) int64 0 1 2 3 4 5 ... 3750 3800 3850 3900 3950 4000
    time             (N_PROF) datetime64[ns] ...
Dimensions without coordinates: N_PROF
Data variables: (12/23)
    TEMP_SURF        (N_PROF, depth) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    PSAL_SURF        (N_PROF, depth) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    TEMP_INTERP      (N_PROF, depth) float64 12.44 12.44 12.44 ... 0.0 0.0 0.0
    PSAL_INTERP      (N_PROF, depth) float64 33.82 33.82 33.82 ... 0.0 0.0 0.0
    DC_REFERENCE     (N_PROF) object b'25563' b'25590' ... b'51533586'
    PLATFORM_NUMBER  (N_PROF) object b'4900190' b'4900191' ... b'AGOR' b'AGOR'
    ...               ...
    SIG              (N_PROF, depth) float64 25.59 25.59 25.59 ... 0.0 0.0 0.0
    MLD              (N_PROF) float64 14.0 22.0 22.0 45.0 ... 60.0 110.0 110.0
    MLD_mask         (N_PROF, depth) float64 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    mask_depthT      (N_PROF, depth) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
    mask_depthS      (N_PROF, depth) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
    depth_prof       (N_PROF) float64 1.9e+03 1.95e+03 2e+03 ... 1.2e+03 1.2e+03

In [3]:
X = np.zeros([len(x['N_PROF']),12])
X[:,0] = x['SLA'].data
X[:,1] = x['latitude'].data
X[:,2] = x['longitude'].data
X[:,3] = x['cos_day'].data
X[:,4] = x['sin_day'].data
X[:,5] = x['MDT'].data
X[:,6] = x['UGOSA'].data
X[:,7] = x['VGOSA'].data
X[:,8] = x['UGOS'].data
X[:,9] = x['VGOS'].data
X[:,10] = x['SST'].data
X[:,11] = -x['bathy'].data

scaler_input = preprocessing.StandardScaler().fit(X)
X_scaled = scaler_input.transform(X)

print(X_scaled.shape)

(87700, 12)


In [4]:
#Load scaler
Sm = load(f'{path_model}/Sm.joblib')
Sstd = load(f'{path_model}/Sstd.joblib')
Tm = load(f'{path_model}/Tm.joblib')
Tstd = load(f'{path_model}/Tstd.joblib')

In [4]:
%%time
#Load models
models_list = glob.glob(f'{path_model}/neuralnet/ensemble/*')
ensemble = []
for model_path in models_list:
    ensemble.append(keras.models.load_model(model_path, compile=False))

CPU times: user 7.58 s, sys: 20 ms, total: 7.6 s
Wall time: 7.57 s


In [6]:
def rmse_from_ae(ae):
    return np.sqrt(np.nanmean((ae ** 2), axis=0))

def get_mean_std_pred(ensemble, X):
    predS = []
    predT = []
    predK = []
    rmse_T = []
    rmse_S = []
    for model in ensemble:
        tmp_pred = model.predict(X)
        temp = tmp_pred[:, :, 0] * Tstd + Tm*X[1]
        psal = tmp_pred[:, :, 1] * Sstd + Sm*X[2]
        predT.append(temp)
        predS.append(psal)
        predK.append(tmp_pred[:, :, 2])
    return np.mean(predT, axis=0), np.std(predT, axis=0), np.mean(predS, axis=0), np.std(predS, axis=0), np.mean(predK,axis=0)

In [7]:
#Predict and store results in x and replace O by NaN
pred_T_mean, pred_T_std, pred_S_mean, pred_S_std, pred_K_mean = get_mean_std_pred(ensemble, [X_scaled, x.mask_depthT.data, x.mask_depthS.data])

#Save Temp_pred and Psal_pred and K_pred and replace 0 by NaN
x = x.assign(variables={"TEMP_predicted": (('N_PROF', 'depth'), pred_T_mean.data)})
temp = np.where(x.TEMP_predicted.data==0, np.nan, x.TEMP_predicted.data)
x = x.assign(variables={"temp_pred": (('N_PROF', 'depth'), temp.data)})

x = x.assign(variables={"PSAL_predicted": (('N_PROF', 'depth'), pred_S_mean.data)})
psal = np.where(x.PSAL_predicted.data==0, np.nan, x.PSAL_predicted.data)
x = x.assign(variables={"psal_pred": (('N_PROF', 'depth'), psal.data)})

x = x.assign(variables={"K_predicted": (('N_PROF', 'depth'), pred_K_mean.data)})
mld = np.where(x.K_predicted.data==0, np.nan, x.K_predicted.data)
x = x.assign(variables={"k_pred": (('N_PROF', 'depth'), mld.data)})

In [8]:
#Compute SIG_pred, save and replace NaN by 0
SA = gsw.SA_from_SP(x['psal_pred'], x['depth'], x['longitude'], x['latitude'])
CT = gsw.CT_from_t(SA, x['temp_pred'], x['depth'])
sigma0 = gsw.sigma0(SA,CT)
x = x.assign(variables={"SIG_predicted": (('N_PROF','depth'), sigma0.data)})
sig_pred = np.where(x.SIG_predicted.data==0, np.nan, x.SIG_predicted.data)
x = x.assign(variables={"sig_pred": (('N_PROF', 'depth'), sig_pred.data)})

#Compute MLD_pred
sig_diff = x.sig_pred - x.sig_pred.sel(depth = 10)-0.03
MLD = x['depth'].where(sig_diff>0).min(dim='depth')
x = x.assign(MLD_pred = MLD)

In [9]:
#Compute MLD from mask K and save
def get_MLD_from_mask(mask):
    mask = np.sign(0.5 - mask)
    return x.depth[np.argmin(mask)]

get_MLD_from_mask_vect = np.vectorize(get_MLD_from_mask, signature='(k)->()')
x = x.assign(variables={"MLD_K": (('N_PROF'), get_MLD_from_mask_vect(x.k_pred))})

#Save the std of temp and psal
x = x.assign(variables={"TEMP_predicted_std": (('N_PROF', 'depth'), pred_T_std.data)})
x = x.assign(variables={"PSAL_predicted_std": (('N_PROF', 'depth'), pred_S_std.data)})

#Retrieve observed temp and psal with NaN
temp = np.where(x.TEMP_INTERP.data==0, np.nan, x.TEMP_INTERP.data)
x = x.assign(variables={"temp": (('N_PROF', 'depth'), temp.data)})
psal = np.where(x.PSAL_INTERP.data==0, np.nan, x.PSAL_INTERP.data)
x = x.assign(variables={"psal": (('N_PROF', 'depth'), psal.data)})
sig = np.where(x.SIG.data==0, np.nan, x.SIG.data)
x = x.assign(variables={"sig": (('N_PROF', 'depth'), sig.data)})

ae_T = np.fabs(x['temp'] - x['temp_pred'])
ae_S = np.fabs(x['psal'] - x['psal_pred'])
x = x.assign(variables={"ae_S": (('N_PROF', 'depth'), ae_S.data)})
x = x.assign(variables={"ae_T": (('N_PROF', 'depth'), ae_T.data)})
  

In [11]:
x.to_netcdf("/home/datawork-lops-bluecloud/osnet/data_out/PRED_alldepth_bathy500_full.nc")